In [62]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps 
#eps is use to avoid  log(0) or 0 in the denominator
from numpy import log2 

In [1]:
#makeing the data set as given in the silde
age = 'youth,youth,middle_aged,senior,senior,senior,middle_aged,youth,youth,senior,youth,middle_aged,middle_aged,senior'.split(',')
income = 'high,high,high,medium,low,low,low,medium,low,medium,medium,medium,high,medium'.split(',')
student = 'no,no,no,no,yes,yes,yes,no,yes,yes,yes,no,yes,no'.split(',')
credit_rating = 'fair,excellent,fair,fair,fair,excellent,excellent,fair,fair,fair,excellent,excellent,fair,excellent'.split(',')
buys_computer = 'no,no,yes,yes,yes,no,yes,no,yes,yes,yes,yes,yes,no'.split(',')


In [64]:
dataset ={'age':age,'income':income,'student':student,'credit_rating':credit_rating,'buys_computer':buys_computer}
df = pd.DataFrame(dataset,columns=['age','income','student','credit_rating','buys_computer'])
df

,age,income,student,credit_rating,buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


![](good_1.jpg)

In [65]:
def info(df):
    last = df.keys()[-1]  #to etract the last which is to be perdicted
    s = 0   #assumed that we have to train on last column only
    values = df[last].unique() #gives all the unique values in last column
    #print(values)
    for value in values:
        pi = df[last].value_counts()[value]/len(df[last])
        s += -pi*np.log2(pi)
    return s
print(info(df))

0.9402859586706311


![](good_2.jpg)

In [66]:
def info_attribute(df,attribute):
    last = df.keys()[-1] 
    target_values = df[last].unique() 
    attribute_values = df[attribute].unique()  #This gives different features in that attribute (like 'youth')
    tot = len(df[attribute]) 
    d=0
    for attribute_value in attribute_values:
        dj=0
        den = len(df[attribute][df[attribute]==attribute_value])
        for target_value in target_values:
            num  = len(df[attribute][df[attribute]==attribute_value][df[last]==target_value])
            pi = num/(den+eps)
            dj+= -pi*np.log2(pi+eps)   #This calculates entropy for one feature like 'youth'
        d+=(den/tot)*dj  #Sums up all the entropy age
    return d

info_attribute(df,df.keys()[0])

0.6935361388961914

In [67]:
#best gain of all the attribute
def best_gain(df):
    max=-1
    attr=0
    info_d = info(df)
    for key in df.keys()[:-1]:
        d_a = info_d-info_attribute(df,key) 
        if(d_a>max):
            max=d_a
            attr=key
    return attr
best_gain(df)

'age'

In [68]:
def buildTree(df):
    target_attribute = df.keys()[-1]
    
    #Get attribute with maximum information gain
    node = best_gain(df)
    attvalue = df[node].unique()
    
    # building decision tree
    #Create an empty dictionary to create tree    
    tree={}
    tree[node]={}
    
    #to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 
    
    for value in attvalue:
        subtable = get_subtable(df,node,value)
        
        clvalue,counts = np.unique(subtable[target_attribute],return_counts=True)     
        
        if len(counts)==1:
            tree[node][value] = clvalue[0]
        else:
            tree[node][value] = buildTree(subtable)
            
    return tree
        
        
        
buildTree(df)    

{'age': {'youth': {'student': {'no': 'no', 'yes': 'yes'}},
  'middle_aged': 'yes',
  'senior': {'credit_rating': {'fair': 'yes', 'excellent': 'no'}}}}

In [69]:
t = buildTree(df)
import pprint
pprint.pprint(t)

{'age': {'middle_aged': 'yes',
         'senior': {'credit_rating': {'excellent': 'no', 'fair': 'yes'}},
         'youth': {'student': {'no': 'no', 'yes': 'yes'}}}}
